In [1]:
import cv2
import pickle
import pandas as pd
import numpy as np
import mediapipe as mp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('Face_keypoints.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z466,v466,x467,y467,z467,v467,x468,y468,z468,v468
0,Surprise,0.524072,0.549330,-0.042529,0.0,0.523366,0.485028,-0.064932,0.0,0.522649,...,-0.000613,0.0,0.609566,0.396959,0.026292,0.0,0.617043,0.386512,0.027607,0.0
1,Surprise,0.528796,0.547702,-0.041729,0.0,0.530803,0.486096,-0.061814,0.0,0.528777,...,0.002040,0.0,0.604993,0.404239,0.028051,0.0,0.611865,0.395554,0.029382,0.0
2,Surprise,0.550601,0.538511,-0.041617,0.0,0.553931,0.478066,-0.061676,0.0,0.550615,...,0.001907,0.0,0.623556,0.400310,0.030076,0.0,0.629978,0.392724,0.031367,0.0
3,Surprise,0.562175,0.549718,-0.044130,0.0,0.568406,0.486052,-0.060482,0.0,0.562831,...,0.007010,0.0,0.630855,0.416757,0.039293,0.0,0.636770,0.408905,0.041209,0.0
4,Surprise,0.583775,0.551414,-0.041062,0.0,0.590257,0.489065,-0.058286,0.0,0.582801,...,0.008185,0.0,0.639132,0.414603,0.047492,0.0,0.644114,0.407629,0.049897,0.0


In [3]:
X = df.drop('class', axis=1)
y = df['class']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print('Train shape:')
print(X_train.shape)
print(X_test.shape)
print('\nTest Shape:')
print(y_train.shape)
print(y_test.shape)

Train shape:
(343, 1872)
(86, 1872)

Test Shape:
(343,)
(86,)


In [6]:
# X_train = StandardScaler().fit_transform(X_train)
# X_test = StandardScaler().fit_transform(X_test)

In [7]:
rc = RandomForestClassifier()

In [8]:
model = rc.fit(X_train, y_train)

In [9]:
y_pred = model.predict(X_test)
y_pred

array(['Happy', 'Surprise', 'Angry', 'Surprise', 'Happy', 'Sad', 'Angry',
       'Angry', 'Happy', 'Angry', 'Happy', 'Surprise', 'Angry', 'Happy',
       'Surprise', 'Angry', 'Happy', 'Surprise', 'Surprise', 'Angry',
       'Sad', 'Sad', 'Happy', 'Surprise', 'Surprise', 'Angry', 'Angry',
       'Happy', 'Sad', 'Happy', 'Angry', 'Happy', 'Sad', 'Angry', 'Happy',
       'Surprise', 'Angry', 'Happy', 'Happy', 'Happy', 'Sad', 'Angry',
       'Surprise', 'Surprise', 'Surprise', 'Angry', 'Sad', 'Surprise',
       'Happy', 'Happy', 'Surprise', 'Sad', 'Angry', 'Surprise',
       'Surprise', 'Sad', 'Sad', 'Surprise', 'Angry', 'Surprise', 'Angry',
       'Sad', 'Surprise', 'Surprise', 'Happy', 'Surprise', 'Happy',
       'Angry', 'Angry', 'Surprise', 'Angry', 'Happy', 'Angry', 'Happy',
       'Happy', 'Happy', 'Surprise', 'Surprise', 'Happy', 'Angry',
       'Angry', 'Happy', 'Surprise', 'Angry', 'Surprise', 'Surprise'],
      dtype=object)

In [23]:
print(accuracy_score(y_test, y_pred))

0.9418604651162791


In [11]:
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [12]:
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [13]:
model

RandomForestClassifier()

In [15]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [22]:
cap = cv2.VideoCapture(1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        

        results = holistic.process(image)

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
#                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
#                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))

        try:
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            row = face_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), 
                                                 [640,480]).astype(int))
            
#             cv2.rectangle(image, (coords[0], coords[1]+5), 
#                           (coords[0]+len(body_language_class)*20, coords[1]-30), (245, 117, 16), -1)
#             cv2.putText(image, body_language_class, coords, 
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            cv2.putText(image, 'CLASS', (95,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (90,40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'PROB', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2)), (10,40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()